In [1]:
import pandas as pd
import numpy as np
from data_generator.batch_generator import BatchGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger
from models import AlexNet
from keras import backend as K
K.set_image_data_format('channels_last')

/home/nicoli/github/alexnet/data_generator/batch_generator.py:42: UserWarning: 'BeautifulSoup' module is missing. The XML-parser will be unavailable.
  warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_classification-100_test_split_47950-70-10-20.csv')

In [3]:
cols = list(df.columns[1:])
in_format = list(df.columns)

In [4]:
train_dataset = BatchGenerator(box_output_format=cols)
validation_dataset = BatchGenerator(box_output_format=cols)

train_dataset.parse_csv(labels_filename='/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_classification-100_train_split_47950-70-10-20.csv', 
                        images_dir='/home/nicoli/github/ssd_keras/dataset/imdb-hand-crop',
                        input_format=in_format)

validation_dataset.parse_csv(labels_filename='/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_classification-100_test_split_47950-70-10-20.csv', 
                             images_dir='/home/nicoli/github/ssd_keras/dataset/imdb-hand-crop',
                             input_format=in_format)

In [5]:
img_height, img_width, img_depth = (224,224,3)

epochs = 100

train_batch_size = 64
shuffle = True
ssd_train = False

validation_batch_size = 32


In [6]:
train_generator = train_dataset.generate(batch_size=train_batch_size,
                                         shuffle=shuffle,
                                         ssd_train=ssd_train,
                                         returns={'processed_labels'},
                                         resize=(img_height, img_width))

validation_generator = validation_dataset.generate(batch_size=validation_batch_size,
                                                   shuffle=shuffle,
                                                   ssd_train=ssd_train,
                                                   returns={'processed_labels'},
                                                   resize=(img_height, img_width))

print("Number of images in the dataset:", train_dataset.get_n_samples())
print("Number of images in the dataset:", validation_dataset.get_n_samples())

Number of images in the dataset: 33565
Number of images in the dataset: 9590


In [7]:
steps_per_epoch = train_dataset.get_n_samples()/train_batch_size
validation_steps = validation_dataset.get_n_samples()/validation_batch_size

In [7]:
alexnet = AlexNet(n_classes=100, img_width=img_width, img_depth=img_depth, img_height=img_height, weights_path='callbacks/alexnet/age/class-weights.05-15.74.hdf5')

In [9]:
alexnet.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 56, 56, 96)        34944     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 56, 56, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 27, 27, 256)       614656    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 27, 27, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 27, 27, 256)       1024      
__________

In [10]:
optimizer = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.00001, amsgrad=True)

In [11]:
csv_logger = CSVLogger('callbacks/alexnet/age/history-classification-01.csv', append=True, separator=',')

checkpoint = ModelCheckpoint(filepath='callbacks/alexnet/age/class-weights.{epoch:02d}-{val_loss:.2f}.hdf5',
                           monitor='val_loss',
                           verbose=1,
                           save_best_only=False,
                           period=1)
#callbacks = [tensorboard, checkpoint]
callbacks=[checkpoint, csv_logger]

In [12]:
alexnet.model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['mse', 'accuracy'])

In [13]:
alexnet.model.fit_generator(train_generator, epochs=epochs, 
                             steps_per_epoch=steps_per_epoch, 
                             validation_data=validation_generator,
                             validation_steps=validation_steps,
                             callbacks=callbacks)

Epoch 1/1
525/524 [==============================] - 4099s 8s/step - loss: 15.6318 - mean_squared_error: 0.0194 - acc: 0.0289 - val_loss: 15.7013 - val_mean_squared_error: 0.0195 - val_acc: 0.0259

Epoch 00001: saving model to callbacks/alexnet/age/class-weights.01-15.70.hdf5
